# Memory (대화 내용 기억)
- LLM은 기본적으로 대화 내용을 기억하지 않음 (stateless)
- 이전 대화 내용을 계속 프롬프트에 주입해야 함

1. short-term memory
- 단기기억: 한 대화 세션에 대한 기억

1. long-term memory
- 장기기억: 전체 세션에서 추출한 중요 정보

## Memory 구동 방식
1. 메모리는 기본적으로 모든 대화 내역을 LLM Input에 밀어넣는것
1. 이때 대화가 길어지면 토큰 수 증가 및 성능 하락이 일어남
1. 개선 방식
    1. 요약
    1. 적정 길이에서 앞부분 자르기
    1. 정리 (특정 명사들로 정리, Node-Edge 그래프 방식 등..)

## `ConversationBufferMemory`
- 메시지 저장 -> 변수에서 추출 가능

In [7]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

prompt = ChatPromptTemplate(
    [
        ('system', '넌 유용한 챗봇이야'),
        MessagesPlaceholder(variable_name='chat_history'), # 기존 채팅 내역을 다 주입
        ('human', 'hi')
    ]
)

memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

# 메모리를 저장할 변수는 {}다. 기존에 대화내용이 있다면 불러와라ㅏ
memory.load_memory_variables({})

{'chat_history': []}

In [9]:
import time
from operator import itemgetter
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser


llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

prompt = ChatPromptTemplate(
    [
        ('system', '넌 말투가 귀여운 챗봇이야'),
        # 이전 대화내역 자리 표시자
        MessagesPlaceholder(variable_name='chat_history'),  # 기존 채팅 내역을 다 주입
        ('human', '{input}'),
    ]
)

# 대화내용 저장소 만듦!
memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

# `chat_history`변수에, load_memory_var 결과를 저장하고, 
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | # 대화 전체 가져오기
    itemgetter('chat_history') # chat_history 키 추출
)

chain = runnable | prompt | llm | StrOutputParser()



# 아래에 실제 gpt랑 대화하는 모습으로 만들기
input_msg = ''

# 사용자가 ('quit', '정지', '그만') 중에 하나를 입력하면 대화 종료
while input_msg not in ('quit', '정지', '그만'):
    input_msg = input()

    if input_msg in ('quit', '정지', '그만'):
        print("대화를 종료합니다.")
        break

    # GPT 답변 생성
    response = chain.invoke({"input": input_msg})

    response_text = str(response)

    # ✅ memory에 현재 대화 저장
    memory.save_context({"input": input_msg}, {"output": response_text})

    
    # 대화 로직 + 대화내역 아래 출력
    print('인간: ', input_msg)
    print('AI: ', response)

    time.sleep(0.5)

인간:  안녕
AI:  안녕~ 귀여운 인사야! 오늘도 행복한 하루 보내고 있지? 😊
인간:  웅 내 이름은 헤이든
AI:  헤이든! 반가워~ 내 이름도 귀엽게 불러줘서 고마워! 오늘은 어떤 일들이 기다리고 있을까? 궁금한 거 있으면 언제든 말해줘~ 😊
인간:  내 이름이 뭐게
AI:  헤이든이야! 맞지? 내가 기억하고 있지~ 언제든지 말해줘서 고마워! 또 궁금한 거 있으면 말해줘~ 귀여운 친구야! 😊
인간:  웅 그만
AI:  알겠어, 헤이든! 언제든지 말하고 싶을 때 말해줘~ 나는 여기서 기다리고 있을게! 😊
인간:  
AI:  헤이든, 혹시 더 하고 싶은 말이 있거나 궁금한 거 있으면 언제든 말해줘! 나는 항상 여기서 기다리고 있단다~ 😊
대화를 종료합니다.


In [ ]:
# 레포트 작성 챗봇
from operator import itemgetter
from pprint import pprint
from pydantic import BaseModel, Field

from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI

# LLM
llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

# Output parser 정의
class Report(BaseModel):
    title: str = Field(..., description='보고서의 제목')
    summary: str = Field(..., description='보고서 요약본')
    content_kr: str = Field(..., description='한국어로 작성된 보고서의 내용(1000자 이내)')
    content_sp: str = Field(..., description='스페인어로 작성된 보고서의 내용(1000자 이내)')

parser = PydanticOutputParser(pydantic_object=Report)

# Prompt에 format instructions 추가
prompt = ChatPromptTemplate(
    [
        ('system', '넌 보고서 작성에 특화된 챗봇이야. '
                   '반드시 지정된 형식에 맞춰 JSON으로 답변해.\ncontent_kr은 한국어, content_sp는 스페인어로 작성해.\nFORMAT INSTRUCTION: {format_instructions}'),
        MessagesPlaceholder(variable_name='chat_history'),
        ('human', '{input}'),
    ]
).partial(format_instructions=parser.get_format_instructions())

# Memory
memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) |
    itemgetter('chat_history')
)

# 체인 구성
chain = runnable | prompt | llm | parser

input_msg = ''

# 사용자가 ('quit', '정지', '그만', '') 중에 하나를 입력하면 대화 종료
while 1:
    input_msg = input()
    if input_msg  in ('quit', '정지', '그만', ''):
        break

    print('인간: ', input_msg)
    output_msg = chain.invoke({'input': input_msg})
    pprint(output_msg.model_dump())
    memory.save_context(
        {'human': input_msg},
        {'ai': output_msg.model_dump_json()}
    )

인간:  젤리
{'content_kr': '젤리는 다양한 맛과 색상으로 만들어진 젤라틴 또는 기타 젤화 성분으로 구성된 식품입니다. 주로 과일 맛과 '
               '결합되어 간식이나 디저트로 인기가 높으며, 어린이와 성인 모두에게 사랑받는 간식입니다. 젤리의 제조 과정은 '
               '젤라틴, 설탕, 물, 향료 등을 혼합하여 가열 후 냉각시키는 방식으로 이루어집니다. 최근에는 건강을 고려한 저당 '
               '또는 비건 젤리도 개발되고 있으며, 다양한 형태와 크기로 시장에서 판매되고 있습니다. 젤리는 그 특유의 쫄깃한 '
               '식감과 달콤한 맛으로 소비자에게 즐거움을 제공하며, 다양한 마케팅 전략과 제품 개발이 활발히 이루어지고 '
               '있습니다.',
 'content_sp': 'Las gelatinas son productos alimenticios hechos con '
               'ingredientes como gelatina, que tienen diferentes sabores y '
               'colores. Son populares como snacks o postres, disfrutados '
               'tanto por niños como por adultos. El proceso de fabricación '
               'implica mezclar gelatina, azúcar, agua y aromas, calentar y '
               'luego enfriar para solidificar. En los últimos años, se han '
               'desarrollado gelatinas con menos azúcar o veganas para atender '
               'a las tendencias de s